# Functions

Topics:
1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

## How to declare a function
Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords

In [ ]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

In [ ]:
function f(x)
    x^2
end

We can call either of these functions like this:

In [ ]:
sayhi("C-3PO")

In [ ]:
f(42)

Alternatively, we could have declared either of these functions in a single line

In [ ]:
sayhi2(name) = println("Hi $name, it's great to see you!")

In [ ]:
f2(x) = x^2

In [ ]:
sayhi2("R2D2")

In [ ]:
f2(42)

Finally, we could have declared these as "anonymous" functions

In [ ]:
sayhi3 = name -> println("Hi $name, it's great to see you!")

In [ ]:
f3 = x -> x^2

In [ ]:
sayhi3("Chewbacca")

In [ ]:
f3(42)

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."* <br><br>
Julia functions will just work on whatever inputs make sense. <br><br>
For example, `sayhi` works on the name of this minor tv character, written as an integer...

In [ ]:
sayhi(55595472)

And `f` will work on a matrix. 

In [ ]:
A = rand(3, 3)
A

In [ ]:
f(A)

`f` will also work on a string like "hi" because `*` is defined for string inputs as string concatenation.

In [ ]:
f("hi")

On the other hand, `f` will not work on a vector. Unlike `A^2`, which is well-defined, the meaning of `v^2` for a vector, `v`, is not a well-defined algebraic operation. 

In [ ]:
v = rand(3)

In [ ]:
f(v)

## Mutating vs. non-mutating functions

As we mentioned before, the convention is that functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.


In [ ]:
v = [3, 5, 2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged. <br><br>

On the other hand, when we run `sort!(v)`, the contents of v are sorted within the array `v`.

In [ ]:
sort!(v)

In [ ]:
v

## Changing names vs. mutation

Recall that variable assignments is just a naming game. Now consider this attempt at a mutating function:

In [1]:
function setzero!(x)
    x = 0
    return x
end
y = 1

1

Do you think this will be able to set `y` to zero? Try it:

In [2]:
setzero!(y)

0

In [3]:
y

1

`y` is just a name for the number `1`, and so was `x` inside the `setzero!` function until we decided it would be better served as a name for `0`! So this function doesn't do any mutation at all — it just returns a `0`. If we want to actually mutate the argument, we need to use indexed assignment (or use a real `!` function that actually does what it claims to do). We also need to use a _mutable_ value as the argument — like an array.

In [ ]:
function setzero!(x)
    x[1] = 0
    return x
end
y = [1]

In [ ]:
setzero!(y)
y

## Some higher order functions

### map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguments. 
`map` then applies that function to every element of the data structure you pass it. For example, executing

```julia
map(f, [1, 2, 3])
```
will give you an output array where the function `f` has been applied to all elements of `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [ ]:
map(f, [1, 2, 3])

Here we've squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as

In [ ]:
x -> x^3

via

In [ ]:
map(x -> x^3, [1, 2, 3])

and now we've cubed all the elements of `[1, 2, 3]`!

### broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do every thing `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`

In [ ]:
broadcast(f, [1, 2, 3])

and again, we've applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`!

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

```julia
broadcast(f, [1, 2, 3])
```
is the same as
```julia
f.([1, 2, 3])
```

In [ ]:
f.([1, 2, 3])

Notice again how different this is from calling 
```julia
f([1, 2, 3])
```
We can square every element of a vector, but we can't square a vector!

To drive home the point, let's look at the difference between

```julia
f(A)
```
and
```julia
f.(A)
```
for a matrix `A`:

In [ ]:
A = [i + 3*j for j in 0:2, i in 1:3]

In [ ]:
f(A)

As before we see that for a matrix, `A`,
```
f(A) = A^2 = A * A
``` 

On the other hand,

In [ ]:
B = f.(A)

contains the squares of all the entries of `A`.

This dot syntax for broadcasting allows us to write relatively complex compound elementwise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

In [ ]:
A .+ 2 .* f.(A) ./ A

instead of

In [ ]:
broadcast(x -> x + 2 * f(x) / x, A)

and the two will perform exactly the same.

# How is broadcast different from map?

With just one argument they're essentially identical, but with more than one argument broadcast has some very special behaviors:

In [4]:
vec = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [5]:
row = [10 20 30 40]

1×4 Matrix{Int64}:
 10  20  30  40

In [6]:
vec .+ row

5×4 Matrix{Int64}:
 11  21  31  41
 12  22  32  42
 13  23  33  43
 14  24  34  44
 15  25  35  45

What just happened!?

In [7]:
repeat(vec, 1, length(row))

5×4 Matrix{Int64}:
 1  1  1  1
 2  2  2  2
 3  3  3  3
 4  4  4  4
 5  5  5  5

In [8]:
repeat(row, length(vec), 1)

5×4 Matrix{Int64}:
 10  20  30  40
 10  20  30  40
 10  20  30  40
 10  20  30  40
 10  20  30  40

In [3]:
A = rand(4, 5)

4×5 Matrix{Float64}:
 0.490542   0.571389  0.0436514  0.476746  0.508673
 0.0471173  0.793234  0.80097    0.624044  0.0290924
 0.143164   0.944371  0.515485   0.889072  0.671755
 0.0344856  0.559584  0.402265   0.679887  0.452504

In [4]:
using Statistics
mean(A, dims=1)

1×5 Matrix{Float64}:
 0.178827  0.717144  0.440593  0.667437  0.415506

In [ ]:
A .- mean(A, dims=1)

In [ ]:
A ./ maximum(A, dims=1)

### Exercises

#### 6.1 
Write a function `add_one` that adds 1 to its input.

In [1]:
add_one = x -> return x + 1;
@assert add_one(1) == 2

In [2]:
@assert add_one(11) == 12

#### 6.2 
Use `map` or `broadcast` to increment every element of matrix `A` by `1` and assign it to a variable `A1`.

In [5]:
A1 = A .+ 1

4×5 Matrix{Float64}:
 1.49054  1.57139  1.04365  1.47675  1.50867
 1.04712  1.79323  1.80097  1.62404  1.02909
 1.14316  1.94437  1.51548  1.88907  1.67176
 1.03449  1.55958  1.40227  1.67989  1.4525

#### 6.3 
Use the broadcast dot syntax to increment every element of matrix `A1` by `1` and store it in variable `A2`

In [6]:
A2 = A1 .+ 1

4×5 Matrix{Float64}:
 2.49054  2.57139  2.04365  2.47675  2.50867
 2.04712  2.79323  2.80097  2.62404  2.02909
 2.14316  2.94437  2.51548  2.88907  2.67176
 2.03449  2.55958  2.40227  2.67989  2.4525

In [ ]:
@assert A2 == [3 4 5; 6 7 8;9 10 11]

Please click on `Validate` on the top, once you are done with the exercises.